# Predicting sentiment from product reviews


The goal of this first notebook is to explore logistic regression and feature engineering with existing GraphLab functions.

In this notebook you will use product review data from Amazon.com to predict whether the sentiments about a product (from its reviews) are positive or negative.

* Use SFrames to do some feature engineering
* Train a logistic regression model to predict the sentiment of product reviews.
* Inspect the weights (coefficients) of a trained logistic regression model.
* Make a prediction (both class and probability) of sentiment for a new product review.
* Given the logistic regression weights, predictors and ground truth labels, write a function to compute the **accuracy** of the model.
* Inspect the coefficients of the logistic regression model and interpret their meanings.
* Compare multiple logistic regression models.

Let's get started!
    
## Fire up GraphLab Create

Make sure you have the latest version of GraphLab Create.

In [1]:
from __future__ import division
# import graphlab
import turicreate as graphlab
import math
import string

# Data preparation

We will use a dataset consisting of baby product reviews on Amazon.com.

In [2]:
products = graphlab.SFrame('../amazon_baby.gl/')

Now, let us see a preview of what the dataset looks like.

In [3]:
products

name,review,rating
Planetwise Flannel Wipes,"These flannel wipes areOK, but in my opinion ...",3.0
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5.0
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5.0
Stop Pacifier Suckingwithout tears with ...,This is a product wellworth the purchase. I ...,5.0
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5.0
Stop Pacifier Suckingwithout tears with ...,"When the Binky Fairy cameto our house, we didn't ...",5.0
A Tale of Baby's Dayswith Peter Rabbit ...,"Lovely book, it's boundtightly so you may no ...",4.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",Perfect for new parents.We were able to keep ...,5.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",A friend of mine pinnedthis product on Pinte ...,5.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",This has been an easy wayfor my nanny to record ...,4.0


## Build the word count vector for each review

Let us explore a specific example of a baby product.


In [4]:
products.column_types

<bound method SFrame.column_types of Columns:
	name	str
	review	str
	rating	float

Rows: 183531

Data:
+-------------------------------+-------------------------------+--------+
|              name             |             review            | rating |
+-------------------------------+-------------------------------+--------+
|    Planetwise Flannel Wipes   | These flannel wipes are OK... |  3.0   |
|     Planetwise Wipe Pouch     | it came early and was not ... |  5.0   |
| Annas Dream Full Quilt wit... | Very soft and comfortable ... |  5.0   |
| Stop Pacifier Sucking with... | This is a product well wor... |  5.0   |
| Stop Pacifier Sucking with... | All of my kids have cried ... |  5.0   |
| Stop Pacifier Sucking with... | When the Binky Fairy came ... |  5.0   |
| A Tale of Baby's Days with... | Lovely book, it's bound ti... |  4.0   |
| Baby Tracker&reg; - Daily ... | Perfect for new parents. W... |  5.0   |
| Baby Tracker&reg; - Daily ... | A friend of mine pinned th... |  5.0  

In [5]:
products[269]

{'name': 'The First Years Massaging Action Teether',
 'review': 'A favorite in our house!',
 'rating': 5.0}

Now, we will perform 2 simple data transformations:

1. Remove punctuation using [Python's built-in](https://docs.python.org/2/library/string.html) string functionality.
2. Transform the reviews into word-counts.

**Aside**. In this notebook, we remove all punctuations for the sake of simplicity. A smarter approach to punctuations would preserve phrases such as "I'd", "would've", "hadn't" and so forth. See [this page](https://www.cis.upenn.edu/~treebank/tokenization.html) for an example of smart handling of punctuations.

In [6]:
def remove_punctuation(text):
    import string
#     return text.translate(None, string.punctuation)
    translator = str.maketrans({key: None for key in string.punctuation})
    return text.translate(translator)

review_without_punctuation = products['review'].apply(remove_punctuation)
products['word_count'] = graphlab.text_analytics.count_words(review_without_punctuation)

Now, let us explore what the sample example above looks like after these 2 transformations. Here, each entry in the **word_count** column is a dictionary where the key is the word and the value is a count of the number of times the word occurs.

In [7]:
products[269]['word_count']

{'our': 1, 'in': 1, 'house': 1, 'favorite': 1, 'a': 1}

In [8]:
products.column_types

<bound method SFrame.column_types of Columns:
	name	str
	review	str
	rating	float
	word_count	dict

Rows: 183531

Data:
+-------------------------------+-------------------------------+--------+
|              name             |             review            | rating |
+-------------------------------+-------------------------------+--------+
|    Planetwise Flannel Wipes   | These flannel wipes are OK... |  3.0   |
|     Planetwise Wipe Pouch     | it came early and was not ... |  5.0   |
| Annas Dream Full Quilt wit... | Very soft and comfortable ... |  5.0   |
| Stop Pacifier Sucking with... | This is a product well wor... |  5.0   |
| Stop Pacifier Sucking with... | All of my kids have cried ... |  5.0   |
| Stop Pacifier Sucking with... | When the Binky Fairy came ... |  5.0   |
| A Tale of Baby's Days with... | Lovely book, it's bound ti... |  4.0   |
| Baby Tracker&reg; - Daily ... | Perfect for new parents. W... |  5.0   |
| Baby Tracker&reg; - Daily ... | A friend of mine pinn

## Extract sentiments

We will **ignore** all reviews with *rating = 3*, since they tend to have a neutral sentiment.

In [9]:
print(len(products[products['rating'] == 3]))
len(products)

16779


183531

In [10]:
products = products[products['rating'] != 3]
len(products)

166752

Now, we will assign reviews with a rating of 4 or higher to be *positive* reviews, while the ones with rating of 2 or lower are *negative*. For the sentiment column, we use +1 for the positive class label and -1 for the negative class label.

In [11]:
products['sentiment'] = products['rating'].apply(lambda rating : +1 if rating > 3 else -1)
products

name,review,rating,word_count,sentiment
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5.0,"{'recommend': 1, 'moist':1, 'osocozy': 1, 'keps': ...",1
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5.0,"{'quilt': 1, 'this': 1,'for': 1, 'anyone': 1, ...",1
Stop Pacifier Suckingwithout tears with ...,This is a product wellworth the purchase. I ...,5.0,"{'tool': 1, 'clever': 1,'little': 1, 'loves': 1, ...",1
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5.0,"{'rock': 1,'headachesthanks': 1, ...",1
Stop Pacifier Suckingwithout tears with ...,"When the Binky Fairy cameto our house, we didn't ...",5.0,"{'thumb': 1, 'or': 1,'break': 1, 'trying': 1, ...",1
A Tale of Baby's Dayswith Peter Rabbit ...,"Lovely book, it's boundtightly so you may no ...",4.0,"{'2995': 1, 'for': 1,'barnes': 1, 'at': 1, ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",Perfect for new parents.We were able to keep ...,5.0,"{'right': 1, 'because':1, 'questions': 1, ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",A friend of mine pinnedthis product on Pinte ...,5.0,"{'like': 1, 'and': 1,'changes': 1, 'the': 1, ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",This has been an easy wayfor my nanny to record ...,4.0,"{'in': 1, 'pages': 1,'out': 1, 'run': 1, ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",I love this journal andour nanny uses it ...,4.0,"{'tracker': 1,'recommend': 1, 'high ...",1


Now, we can see that the dataset contains an extra column called **sentiment** which is either positive (+1) or negative (-1).

## Split data into training and test sets

Let's perform a train/test split with 80% of the data in the training set and 20% of the data in the test set. We use `seed=1` so that everyone gets the same result.

In [12]:
train_data, test_data = products.random_split(.8, seed=1)
print(len(train_data))
print(len(test_data))

133416
33336


# Train a sentiment classifier with logistic regression

We will now use logistic regression to create a sentiment classifier on the training data. This model will use the column **word_count** as a feature and the column **sentiment** as the target. We will use `validation_set=None` to obtain same results as everyone else.

**Note:** This line may take 1-2 minutes.

In [13]:
sentiment_model = graphlab.logistic_classifier.create(train_data,
                                                      target = 'sentiment',
                                                      features=['word_count'],
                                                      validation_set=None)

Logistic regression:

--------------------------------------------------------

Number of examples          : 133416

Number of classes           : 2

Number of feature columns   : 1

...

| 5         | 12       | 3.000000  | 2.915360     | 0.960964          |

| 6         | 13       | 3.000000  | 3.104491     | 0.975033          |

+-----------+----------+-----------+--------------+-------------------+

TERMINATED: Terminated due to numerical difficulties.

This model may not be ideal. To improve it, consider doing one of the following:
(a) Increasing the regularization.
(b) Standardizing the input data.
(c) Removing highly correlated features.
(d) Removing `inf` and `NaN` values in the training data.

In [14]:
sentiment_model

Class                          : LogisticClassifier

Schema
------
Number of coefficients         : 121713
Number of examples             : 133416
Number of classes              : 2
Number of feature columns      : 1
Number of unpacked features    : 121712

Hyperparameters
---------------
L1 penalty                     : 0.0
L2 penalty                     : 0.01

Training Summary
----------------
Solver                         : lbfgs
Solver iterations              : 6
Solver status                  : TERMINATED: Terminated due to numerical difficulties.
Training time (sec)            : 3.283

Settings
--------
Log-likelihood                 : inf

Highest Positive Coefficients
-----------------------------
word_count[mobileupdate]       : 41.9847
word_count[placeid]            : 41.7354
word_count[labelbox]           : 41.151
word_count[httpwwwamazoncomreviewrhgg6qp7tdnhbrefcmcrprcmtieutf8asinb00318cla0nodeid] : 40.0454
word_count[knobskeeping]       : 36.2091

Lowest Negative Coeffic

**Aside**. You may get a warning to the effect of "Terminated due to numerical difficulties --- this model may not be ideal". It means that the quality metric (to be covered in Module 3) failed to improve in the last iteration of the run. The difficulty arises as the sentiment model puts too much weight on extremely rare words. A way to rectify this is to apply regularization, to be covered in Module 4. Regularization lessens the effect of extremely rare words. For the purpose of this assignment, however, please proceed with the model above.

Now that we have fitted the model, we can extract the weights (coefficients) as an SFrame as follows:

In [16]:
weights = sentiment_model.coefficients
weights.column_names()

['name', 'index', 'class', 'value', 'stderr']

There are a total of `121713` coefficients in the model. Recall from the lecture that positive weights $w_j$ correspond to weights that cause positive sentiment, while negative weights correspond to negative sentiment. 

Fill in the following block of code to calculate how many *weights* are positive ( >= 0). (**Hint**: The `'value'` column in SFrame *weights* must be positive ( >= 0)).

In [17]:
print(len(weights['value']))

121713


In [19]:
num_positive_weights = len(weights[weights['value'] >= 0])
num_negative_weights = len(weights) - num_positive_weights

print("Number of positive weights: %s " % num_positive_weights)
print("Number of negative weights: %s " % num_negative_weights)

Number of positive weights: 68419 
Number of negative weights: 53294 


**Quiz Question:** How many weights are >= 0?

## Making predictions with logistic regression

Now that a model is trained, we can make predictions on the **test data**. In this section, we will explore this in the context of 3 examples in the test dataset.  We refer to this set of 3 examples as the **sample_test_data**.

In [20]:
sample_test_data = test_data[10:13]
print(sample_test_data['rating'])
sample_test_data

[5.0, 2.0, 1.0]


name,review,rating,word_count,sentiment
Our Baby Girl Memory Book,Absolutely love it andall of the Scripture in ...,5.0,"{'again': 1, 'book': 1,'same': 1, 'receive': 1, ...",1
Wall Decor RemovableDecal Sticker - Colorful ...,Would not purchase againor recommend. The decals ...,2.0,"{'peeling': 1, '5': 1,'about': 1, 'for': 1, ...",-1
New Style Trailing CherryBlossom Tree Decal ...,Was so excited to getthis product for my baby ...,1.0,"{'on': 1, 'waste': 1,'wouldnt': 1, ...",-1


Let's dig deeper into the first row of the **sample_test_data**. Here's the full review:

In [21]:
sample_test_data[0]['review']

'Absolutely love it and all of the Scripture in it.  I purchased the Baby Boy version for my grandson when he was born and my daughter-in-law was thrilled to receive the same book again.'

That review seems pretty positive.

Now, let's see what the next row of the **sample_test_data** looks like. As we could guess from the sentiment (-1), the review is quite negative.

In [22]:
sample_test_data[1]['review']

'Would not purchase again or recommend. The decals were thick almost plastic like and were coming off the wall as I was applying them! The would NOT stick! Literally stayed stuck for about 5 minutes then started peeling off.'

In [23]:
sample_test_data[2]['review']

"Was so excited to get this product for my baby girls bedroom!  When I got it the back is NOT STICKY at all!  Every time I walked into the bedroom I was picking up pieces off of the floor!  Very very frustrating!  Ended up having to super glue it to the wall...very disappointing.  I wouldn't waste the time or money on it."

We will now make a **class** prediction for the **sample_test_data**. The `sentiment_model` should predict **+1** if the sentiment is positive and **-1** if the sentiment is negative. Recall from the lecture that the **score** (sometimes called **margin**) for the logistic regression model  is defined as:

$$
\mbox{score}_i = \mathbf{w}^T h(\mathbf{x}_i)
$$ 

where $h(\mathbf{x}_i)$ represents the features for example $i$.  We will write some code to obtain the **scores** using GraphLab Create. For each row, the **score** (or margin) is a number in the range **[-inf, inf]**.

In [24]:
scores = sentiment_model.predict(sample_test_data, output_type='margin')
print(scores)

[6.7346197271094805, -5.734130996789538, -14.668460404504883]


### Predicting sentiment

These scores can be used to make class predictions as follows:

$$
\hat{y} = 
\left\{
\begin{array}{ll}
      +1 & \mathbf{w}^T h(\mathbf{x}_i) > 0 \\
      -1 & \mathbf{w}^T h(\mathbf{x}_i) \leq 0 \\
\end{array} 
\right.
$$

Using scores, write code to calculate $\hat{y}$, the class predictions:

In [25]:
def my_preds(scores):
    return scores.apply(lambda x: +1 if x > 0 else -1)


my_pr = my_preds(scores)
my_pr

dtype: int
Rows: 3
[1, -1, -1]

Run the following code to verify that the class predictions obtained by your calculations are the same as that obtained from GraphLab Create.

In [26]:
print("Class predictions according to GraphLab Create:")
print(sentiment_model.predict(sample_test_data))

Class predictions according to GraphLab Create:
[1, -1, -1]


**Checkpoint**: Make sure your class predictions match with the one obtained from GraphLab Create.

### Probability predictions

Recall from the lectures that we can also calculate the probability predictions from the scores using:
$$
P(y_i = +1 | \mathbf{x}_i,\mathbf{w}) = \frac{1}{1 + \exp(-\mathbf{w}^T h(\mathbf{x}_i))}.
$$

Using the variable **scores** calculated previously, write code to calculate the probability that a sentiment is positive using the above formula. For each row, the probabilities should be a number in the range **[0, 1]**.

In [27]:
def my_proba(scores):
    return scores.apply(lambda s: 1./(1. + math.exp(-s)))

my_pr = my_proba(scores)
my_pr

dtype: float
Rows: 3
[0.9988123848377792, 0.0032232681817066055, 4.261557996500454e-07]

**Checkpoint**: Make sure your probability predictions match the ones obtained from GraphLab Create.

In [28]:
print("Class predictions according to GraphLab Create:")
print(sentiment_model.predict(sample_test_data, output_type='probability'))

Class predictions according to GraphLab Create:
[0.9988123848377793, 0.003223268181706607, 4.261557996500451e-07]


** Quiz Question:** Of the three data points in **sample_test_data**, which one (first, second, or third) has the **lowest probability** of being classified as a positive review?

# Find the most positive (and negative) review

We now turn to examining the full test dataset, **test_data**, and use GraphLab Create to form predictions on all of the test data points for faster performance.

Using the `sentiment_model`, find the 20 reviews in the entire **test_data** with the **highest probability** of being classified as a **positive review**. We refer to these as the "most positive reviews."

To calculate these top-20 reviews, use the following steps:
1.  Make probability predictions on **test_data** using the `sentiment_model`. (**Hint:** When you call `.predict` to make predictions on the test data, use option `output_type='probability'` to output the probability rather than just the most likely class.)
2.  Sort the data according to those predictions and pick the top 20. (**Hint:** You can use the `.topk` method on an SFrame to find the top k rows sorted according to the value of a specified column.)

In [29]:
test_data

name,review,rating,word_count,sentiment
"Baby Tracker&reg; - DailyChildcare Journal, ...",This has been an easy wayfor my nanny to record ...,4.0,"{'in': 1, 'pages': 1,'out': 1, 'run': 1, ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",I love this journal andour nanny uses it ...,4.0,"{'tracker': 1,'recommend': 1, 'high ...",1
Nature's Lullabies FirstYear Sticker Calendar ...,"I love this littlecalender, you can keep ...",5.0,"{'too': 1, 'stickers': 1,'illustrations': 1, ...",1
Nature's Lullabies SecondYear Sticker Calendar ...,"I had a hard time findinga second year calendar, ...",5.0,"{'reference': 1, 'have':1, 'out': 1, 'fill': 1, ...",1
"Lamaze Peekaboo, I LoveYou ...","One of baby's first andfavorite books, and i ...",4.0,"{'typical': 1, 'your': 1,'the': 1, 'family': 1, ...",1
"Lamaze Peekaboo, I LoveYou ...",My son loved this book asan infant. It was ...,5.0,"{'farm': 1, 'out': 1,'say': 1, 'again': 1, ...",1
"Lamaze Peekaboo, I LoveYou ...",Our baby loves this book& has loved it for a ...,5.0,"{'own': 1, 'his': 1,'and': 1, 'now': 1, ' ...",1
"SoftPlay Giggle JiggleFunbook, Happy Bear ...",This bear is absolutelyadorable and I would ...,2.0,"{'kenzie': 1, 'my': 1,'down': 1, 'gift': 1, ...",-1
SoftPlay Peek-A-BooWhere's Elmo A Childr ...,I bought two for recentbaby showers! The book ...,5.0,"{'doesnt': 1, 'who': 1,'elmo': 1, 'for': 2, ...",1
Baby's First Year UndatedWall Calendar with ...,I searched high and lowfor a first year cale ...,5.0,"{'pictures': 1,'personalization': 1, ...",1


In [30]:
test_data['pred_proba'] = sentiment_model.predict(test_data, output_type='probability')
test_data['name', 'pred_proba'].topk('pred_proba', 20).print_rows(num_rows=20, num_columns=10) 

+-------------------------------+------------+
|              name             | pred_proba |
+-------------------------------+------------+
|   Munchkin Mozart Magic Cube  |    1.0     |
|  BABYBJORN Potty Chair - Red  |    1.0     |
| Safety 1st Tot-Lok Four Lo... |    1.0     |
| Summer Infant Complete Nur... |    1.0     |
| Leachco Snoogle Total Body... |    1.0     |
| HALO SleepSack Micro-Fleec... |    1.0     |
| Peg Perego Primo Viaggio C... |    1.0     |
|   Capri Stroller - Red Tech   |    1.0     |
| Wizard Convertible Car Sea... |    1.0     |
| Britax Marathon Convertibl... |    1.0     |
| Britax Decathlon Convertib... |    1.0     |
| North States Supergate Pre... |    1.0     |
|  Fisher-Price Deluxe Jumperoo |    1.0     |
| Lilly Gold Sit 'n' Stroll ... |    1.0     |
| Fisher-Price Rainforest Me... |    1.0     |
| JP Lizzy Chocolate Ice Cla... |    1.0     |
| Cloud b Sound Machine Soot... |    1.0     |
| Traveling Toddler Car Seat... |    1.0     |
| Ameda Purel

**Quiz Question**: Which of the following products are represented in the 20 most positive reviews? [multiple choice]


Now, let us repeat this exercise to find the "most negative reviews." Use the prediction probabilities to find the  20 reviews in the **test_data** with the **lowest probability** of being classified as a **positive review**. Repeat the same steps above but make sure you **sort in the opposite order**.

In [33]:
test_data['name', 'pred_proba'].sort('pred_proba').print_rows(num_rows=20, num_columns=10) 

+-------------------------------+------------------------+
|              name             |       pred_proba       |
+-------------------------------+------------------------+
| Jolly Jumper Arctic Sneak ... | 7.804150680814363e-100 |
| Levana Safe N'See Digital ... | 6.836508853726986e-25  |
| Snuza Portable Baby Moveme... | 2.126545107527289e-24  |
| Fisher-Price Ocean Wonders... | 2.245820807310098e-23  |
| VTech Communications Safe ... | 1.3296296617357631e-22 |
| Safety 1st High-Def Digita... | 2.068720974535566e-20  |
| Chicco Cortina KeyFit 30 T... | 5.938819945184121e-20  |
| Prince Lionheart Warmies W... | 6.285100162744517e-20  |
| Valco Baby Tri-mode Twin S... |  8.05528712312116e-20  |
| Adiri BPA Free Natural Nur... | 8.465217243658139e-20  |
| Munchkin Nursery Projector... | 1.5285394512981903e-19 |
| The First Years True Choic... | 1.7790188977383146e-19 |
| Nuby Natural Touch Silicon... | 1.1522735383237443e-18 |
| Peg-Perego Tatamia High Ch... | 1.2617566612125727e-18

**Quiz Question**: Which of the following products are represented in the 20 most negative reviews?  [multiple choice]

## Compute accuracy of the classifier

We will now evaluate the accuracy of the trained classifier. Recall that the accuracy is given by


$$
\mbox{accuracy} = \frac{\mbox{# correctly classified examples}}{\mbox{# total examples}}
$$

This can be computed as follows:

* **Step 1:** Use the trained model to compute class predictions (**Hint:** Use the `predict` method)
* **Step 2:** Count the number of data points when the predicted class labels match the ground truth labels (called `true_labels` below).
* **Step 3:** Divide the total number of correct predictions by the total number of data points in the dataset.

Complete the function below to compute the classification accuracy:

In [34]:
def get_classification_accuracy(model, data, true_labels):
    # First get the predictions
    ## YOUR CODE HERE
    data['pred_sentiment'] = model.predict(data)
    
    # Compute the number of correctly classified examples
    ## YOUR CODE HERE
    correct_samples = len(data[data['sentiment'] == data['pred_sentiment']])

    # Then compute accuracy by dividing num_correct by total number of examples
    ## YOUR CODE HERE
    accuracy = correct_samples/len(data)
    
    return accuracy

Now, let's compute the classification accuracy of the **sentiment_model** on the **test_data**.

In [35]:
get_classification_accuracy(sentiment_model, test_data, test_data['sentiment'])

0.9145368370530358

**Quiz Question**: What is the accuracy of the **sentiment_model** on the **test_data**? Round your answer to 2 decimal places (e.g. 0.76).

**Quiz Question**: Does a higher accuracy value on the **training_data** always imply that the classifier is better?

## Learn another classifier with fewer words

There were a lot of words in the model we trained above. We will now train a simpler logistic regression model using only a subset of words that occur in the reviews. For this assignment, we selected a 20 words to work with. These are:

In [36]:
significant_words = ['love', 'great', 'easy', 'old', 'little', 'perfect', 'loves', 
      'well', 'able', 'car', 'broke', 'less', 'even', 'waste', 'disappointed', 
      'work', 'product', 'money', 'would', 'return']

In [37]:
len(significant_words)

20

For each review, we will use the **word_count** column and trim out all words that are **not** in the **significant_words** list above. We will use the [SArray dictionary trim by keys functionality]( https://dato.com/products/create/docs/generated/graphlab.SArray.dict_trim_by_keys.html). Note that we are performing this on both the training and test set.

In [38]:
train_data['word_count_subset'] = train_data['word_count'].dict_trim_by_keys(significant_words, exclude=False)
test_data['word_count_subset'] = test_data['word_count'].dict_trim_by_keys(significant_words, exclude=False)

Let's see what the first example of the dataset looks like:

In [39]:
train_data[0]['review']

'it came early and was not disappointed. i love planet wise bags and now my wipe holder. it keps my osocozy wipes moist and does not leak. highly recommend it.'

The **word_count** column had been working with before looks like the following:

In [41]:
print(train_data[0]['word_count'])

{'recommend': 1, 'moist': 1, 'osocozy': 1, 'keps': 1, 'leak': 1, 'holder': 1, 'was': 1, 'now': 1, 'wipe': 1, 'my': 2, 'disappointed': 1, 'wise': 1, 'and': 3, 'planet': 1, 'it': 3, 'love': 1, 'early': 1, 'not': 2, 'highly': 1, 'wipes': 1, 'bags': 1, 'came': 1, 'does': 1, 'i': 1}


Since we are only working with a subset of these words, the column **word_count_subset** is a subset of the above dictionary. In this example, only 2 `significant words` are present in this review.

In [43]:
print(train_data[0]['word_count_subset'])

{'disappointed': 1, 'love': 1}


## Train a logistic regression model on a subset of data

We will now build a classifier with **word_count_subset** as the feature and **sentiment** as the target. 

In [44]:
simple_model = graphlab.logistic_classifier.create(train_data,
                                                   target = 'sentiment',
                                                   features=['word_count_subset'],
                                                   validation_set=None)
simple_model

Logistic regression:

--------------------------------------------------------

Number of examples          : 133416

Number of classes           : 2

Number of feature columns   : 1

...

| 5         | 6        | 0.229749     | 0.866815          |

| 6         | 7        | 0.263675     | 0.866815          |

+-----------+----------+--------------+-------------------+

SUCCESS: Optimal solution found.

Class                          : LogisticClassifier

Schema
------
Number of coefficients         : 21
Number of examples             : 133416
Number of classes              : 2
Number of feature columns      : 1
Number of unpacked features    : 20

Hyperparameters
---------------
L1 penalty                     : 0.0
L2 penalty                     : 0.01

Training Summary
----------------
Solver                         : newton
Solver iterations              : 6
Solver status                  : SUCCESS: Optimal solution found.
Training time (sec)            : 0.274

Settings
--------
Log-likelihood                 : 44323.7254

Highest Positive Coefficients
-----------------------------
word_count_subset[loves]       : 1.6773
word_count_subset[perfect]     : 1.5145
word_count_subset[love]        : 1.3654
(intercept)                    : 1.2995
word_count_subset[easy]        : 1.1937

Lowest Negative Coefficients
----------------------------
word_count_subset[disappointed] : -2.3551
wor

We can compute the classification accuracy using the `get_classification_accuracy` function you implemented earlier.

In [45]:
get_classification_accuracy(simple_model, test_data, test_data['sentiment'])

0.8693004559635229

Now, we will inspect the weights (coefficients) of the **simple_model**:

In [47]:
simple_model.coefficients

name,index,class,value,stderr
(intercept),None,1,1.2995449552027034,0.012088854133041695
word_count_subset,disappointed,1,-2.3550925006107253,0.05041498885568342
word_count_subset,love,1,1.3654354936790376,0.030354629510850255
word_count_subset,little,1,0.520628636025019,0.02146914756647254
word_count_subset,loves,1,1.6772714555592905,0.048232827538257236
word_count_subset,product,1,-0.32055549299557495,0.01543113213619574
word_count_subset,well,1,0.5042567463979287,0.021381300630975462
word_count_subset,great,1,0.9446912694798453,0.020950992659027497
word_count_subset,easy,1,1.1936618983284653,0.029288869201985937
word_count_subset,work,1,-0.6217000124253141,0.02303305979457698


Let's sort the coefficients (in descending order) by the **value** to obtain the coefficients with the most positive effect on the sentiment.

In [48]:
simple_model.coefficients.sort('value', ascending=False).print_rows(num_rows=21)

+-------------------+--------------+-------+----------------------+
|        name       |    index     | class |        value         |
+-------------------+--------------+-------+----------------------+
| word_count_subset |    loves     |   1   |  1.6772714555592905  |
| word_count_subset |   perfect    |   1   |  1.5144862670271348  |
| word_count_subset |     love     |   1   |  1.3654354936790376  |
|    (intercept)    |     None     |   1   |  1.2995449552027034  |
| word_count_subset |     easy     |   1   |  1.1936618983284653  |
| word_count_subset |    great     |   1   |  0.9446912694798453  |
| word_count_subset |    little    |   1   |  0.520628636025019   |
| word_count_subset |     well     |   1   |  0.5042567463979287  |
| word_count_subset |     able     |   1   | 0.19143830229475103  |
| word_count_subset |     old      |   1   | 0.08539618866781597  |
| word_count_subset |     car      |   1   | 0.05883499006802043  |
| word_count_subset |     less     |   1   | -0.

**Quiz Question**: Consider the coefficients of **simple_model**. There should be 21 of them, an intercept term + one for each word in **significant_words**. How many of the 20 coefficients (corresponding to the 20 **significant_words** and *excluding the intercept term*) are positive for the `simple_model`?

In [53]:
assert 10 == len(simple_model.coefficients[simple_model.coefficients['value']>0]) - 1 # remove the intercept

**Quiz Question**: Are the positive words in the **simple_model** (let us call them `positive_significant_words`) also positive words in the **sentiment_model**?

In [71]:
positive_significant_words = simple_model.coefficients[simple_model.coefficients['value']>0]['index']
positive_significant_words = list(positive_significant_words.to_numpy()[1:])
positive_significant_words

['love',
 'little',
 'loves',
 'well',
 'great',
 'easy',
 'able',
 'perfect',
 'old',
 'car']

In [78]:
sentiment_model.coefficients['index']

dtype: str
Rows: 121713
[None, 'recommend', 'moist', 'osocozy', 'keps', 'leak', 'holder', 'was', 'now', 'wipe', 'my', 'disappointed', 'wise', 'and', 'planet', 'it', 'love', 'early', 'not', 'highly', 'wipes', 'bags', 'came', 'does', 'i', 'quilt', 'this', 'for', 'anyone', 'to', 'perfectlywould', 'of', 'bed', 'the', 'than', 'comfortable', 'full', 'warmer', 'size', 'type', 'very', 'looksfit', 'looking', 'soft', 'tool', 'clever', 'little', 'loves', 'a', 'losing', 'proud', 'she', 'rid', 'in', 'daughter', 'artwork', 'purchase', 'ownership', 'much', 'how', 'about', 'product', 'chart', 'her', 'what', 'well', 'has', 'binky', 'approach', 'positive', 'found', 'getting', 'like', 'have', 'worth', 'ingenious', 'most', 'so', 'else', 'fairy', 'anything', 'back', 'herself', 'is', 'rock', 'headachesthanks', 'many', 'soo', 'will', 'expecting', 'where', 'until', 'great', 'going', 'itthis', 'must', 'understand', 'from', 'part', 'help', ... ]

In [79]:
sentiment_model.coefficients.filter_by(positive_significant_words, 'index')

name,index,class,value,stderr
word_count,love,1,1.433016854407187,None
word_count,little,1,0.6741624575058623,None
word_count,loves,1,1.5664851757106897,None
word_count,well,1,0.6279648775709666,None
word_count,great,1,1.3145924503979616,None
word_count,easy,1,1.2134693782267674,None
word_count,able,1,0.17433127255300637,None
word_count,perfect,1,1.7519011439369414,None
word_count,old,1,0.009122301134896618,None
word_count,car,1,0.19526367061946717,None


# Comparing models

We will now compare the accuracy of the **sentiment_model** and the **simple_model** using the `get_classification_accuracy` method you implemented above.

First, compute the classification accuracy of the **sentiment_model** on the **train_data**:

In [80]:
get_classification_accuracy(sentiment_model, train_data, train_data['sentiment'])

0.979440247046831

Now, compute the classification accuracy of the **simple_model** on the **train_data**:

In [81]:
get_classification_accuracy(simple_model, train_data, train_data['sentiment'])

0.8668150746537147

**Quiz Question**: Which model (**sentiment_model** or **simple_model**) has higher accuracy on the TRAINING set?

Now, we will repeat this exercise on the **test_data**. Start by computing the classification accuracy of the **sentiment_model** on the **test_data**:

In [82]:
get_classification_accuracy(sentiment_model, test_data, test_data['sentiment'])

0.9145368370530358

Next, we will compute the classification accuracy of the **simple_model** on the **test_data**:

In [83]:
get_classification_accuracy(simple_model, test_data, test_data['sentiment'])

0.8693004559635229

**Quiz Question**: Which model (**sentiment_model** or **simple_model**) has higher accuracy on the TEST set?

## Baseline: Majority class prediction

It is quite common to use the **majority class classifier** as the a baseline (or reference) model for comparison with your classifier model. The majority classifier model predicts the majority class for all data points. At the very least, you should healthily beat the majority class classifier, otherwise, the model is (usually) pointless.

What is the majority class in the **train_data**?

In [85]:
num_positive  = (train_data['sentiment'] == +1).sum()
num_negative = (train_data['sentiment'] == -1).sum()
print(num_positive)
print(num_negative)

112164
21252


Now compute the accuracy of the majority class classifier on **test_data**.

**Quiz Question**: Enter the accuracy of the majority class classifier model on the **test_data**. Round your answer to two decimal places (e.g. 0.76).

In [88]:
num_positive  = (test_data['sentiment'] == +1).sum()
num_negative = (test_data['sentiment'] == -1).sum()
print(num_positive)
print(num_negative)
print(len(test_data))

accuracy = num_positive/(num_positive+num_negative)
accuracy

28095
5241
33336


0.8427825773938085

**Quiz Question**: Is the **sentiment_model** definitely better than the majority class classifier (the baseline)?